In [1]:
import os
import sys

# Get the absolute path of the backend folder
backend_path = os.path.abspath(os.path.join(os.getcwd(), "..", "backend"))

# Add backend to sys.path
if backend_path not in sys.path:
    sys.path.append(backend_path)

In [43]:
# MAIN IMPORTS
import uuid
import json
import nest_asyncio

import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, TimestampType
from pyspark.sql.functions import col

from scripts.scraper import scrape_medium

nest_asyncio.apply()

In [18]:
# Create a Spark session with ClickHouse JDBC
spark = SparkSession.builder \
    .appName("ClickHouse-Spark") \
    .config("spark.driver.extraClassPath", "/opt/spark/jars/*") \
    .config("spark.executor.extraClassPath", "/opt/spark/jars/*") \
    .getOrCreate()

# ClickHouse connection details
CLICKHOUSE_URL = "jdbc:clickhouse://clickhouse:8123/medium"
CLICKHOUSE_DRIVER = "com.clickhouse.jdbc.ClickHouseDriver"

print("Spark session successfully started and connected to ClickHouse!")

Spark session successfully started and connected to ClickHouse!


In [19]:
# Define the table to read from
table_name = "medium_articles"

# Read data into a Spark DataFrame
df = spark.read \
    .format("jdbc") \
    .option("url", CLICKHOUSE_URL) \
    .option("dbtable", table_name) \
    .option("driver", CLICKHOUSE_DRIVER) \
    .option("user", "default") \
    .option("password", "your_password") \
    .load()

# Show the first few rows
df.show()

+---+-----+--------------+-------+-----+--------+----+----------+----------+
| id|title|published_date|paywall|claps|comments|link|topic_name|topic_type|
+---+-----+--------------+-------+-----+--------+----+----------+----------+
+---+-----+--------------+-------+-----+--------+----+----------+----------+



In [5]:
data = scrape_medium("technology/recommended")

In [10]:
df_free = pd.DataFrame([x for x in data if x["paywall"] == False])

In [14]:
df_free.sort_values(by="claps", ascending=False).head()

,title,link,paywall,published_date,claps,comments,topic_name,topic_type,objs
17,Cracking the code of vibe coding,https://medium.com/user-experience-design-1/cr...,False,2025-03-21 00:00:00.000000,1200,26,technology,recommended,"[Mar 211.2K26, Mar 21, 1.2K, 26, ]"
1,Acids to Ashes: The crash of Pan Am flight 160,https://medium.com/@admiralcloudberg/acids-to-...,False,2025-03-28 16:29:59.907120,921,15,technology,recommended,"[2d ago92115, 921, 15, ]"
12,Everything You’ve Heard About AI In Game Devel...,https://medium.com/@sa-liberty/everything-youv...,False,2025-03-23 00:00:00.000000,746,16,technology,recommended,"[Mar 2374616, Mar 23, 746, 16, ]"
6,It’s all fun and games… until your boss starts...,https://medium.com/user-experience-design-1/it...,False,2025-03-27 16:29:59.909872,512,13,technology,recommended,"[3d ago51213, 512, 13, ]"
8,We Ruined Our Minds,https://medium.com/@newalbrecht/we-ruined-our-...,False,2025-03-26 16:29:59.910774,510,9,technology,recommended,"[4d ago5109, 510, 9, ]"


In [36]:
for record in data:
    record["id"] = str(uuid.uuid4())

In [44]:
# Convert JSON data to a DataFrame
medium_articles_schema = StructType([
    StructField("id", StringType(), False),  # UUID stored as string in Spark
    StructField("title", StringType(), False),
    StructField("published_date", TimestampType(), True),
    StructField("paywall", BooleanType(), True),
    StructField("claps", IntegerType(), True),
    StructField("comments", IntegerType(), True),
    StructField("link", StringType(), True),
    StructField("topic_name", StringType(), True),
    StructField("topic_type", StringType(), True),
    StructField("version", IntegerType(), True)
])

df = spark.createDataFrame(data, schema=medium_articles_schema).withColumn("id", col("id").cast("string"))

In [45]:
df.show()

+--------------------+--------------------+--------------------+-------+-----+--------+--------------------+----------+-----------+
|                  id|               title|      published_date|paywall|claps|comments|                link|topic_name| topic_type|
+--------------------+--------------------+--------------------+-------+-----+--------+--------------------+----------+-----------+
|7dfe6695-600a-424...|Google Hasn’t Fig...| 2025-03-21 00:00:00|   true|   83|       2|https://medium.co...|technology|recommended|
|a8fda7d3-62ef-4ed...|Which Countries H...|2025-03-28 16:29:...|   true|   61|       2|https://medium.co...|technology|recommended|
|7f2ee5d1-43ec-422...|LaMBDA and the Lo...|2025-03-28 16:29:...|  false|  103|       2|https://medium.co...|technology|recommended|
|0d58d7d9-fa11-46f...|Acids to Ashes: T...|2025-03-28 16:29:...|  false|  921|      15|https://medium.co...|technology|recommended|
|2bb5af55-eabf-40f...|Is this the death...|2025-03-28 16:29:...|   true|  23

In [46]:
# Write DataFrame to ClickHouse
df.write \
    .format("jdbc") \
    .option("url", CLICKHOUSE_URL) \
    .option("dbtable", "medium_articles") \
    .option("user", "default") \
    .option("password", "your_password") \
    .option("driver", "com.clickhouse.jdbc.ClickHouseDriver") \
    .mode("append") \
    .save()